In [ ]:
import datetime as dt
import json
import os
import subprocess
import time
from glob import glob

import requests
from config import SETTINGS
from geo.Geoserver import Geoserver
from tqdm.notebook import tqdm

# Upload NIVA datasets

Files within the `niva` bucket are arranged in a fairly complex hierarchy. This can probably be simplified, but it's going to take a while.

This notebook searches a "mission" folder for orthomosaics with user-specified names. For example, Hege sent me a list of mosaic names for the Kelpmap project (which I hadn't previously found because they're quite deeply buried). The orginal file names are generally not very helpful, so in this notebook the user must specify more suitable names for each file. This is done manually. The code then performs the following operations:

 1. Finds the files on MinIO
 2. Builds 3-band COGs for all datasets
 3. Uploads the files to GeoServer
 4. Publishes them to GeoNode
 5. Updates the basic metadata by extracting information from the user-specified file names
 
## 1. Find files

In [ ]:
def find_file_by_name(fname, parent_folder):
    result = []
    for root, dirs, files in os.walk(parent_folder):
        if fname in files:
            result.append(os.path.join(root, fname))
    return result

In [ ]:
# Mission folder to search
base_dir = r"/home/notebook/shared-seabee-ns9879k/niva/2022/2022-08-18_KELPMAP_Vega/"

# Input files to search for, mapped to more helpful output file names:
#     project_region_area_org_spec_date-time.tif
fnames_dict = {
    "KelpMap_N_20220819_MS_comp.tif": "KELPMAP_Vega_North_Spectrofly_MS_20220819-0000.tif",
    "20220819_1230_RGB_120m_transparent_mosaic_group1.tif": "KELPMAP_Vega_North_Spectrofly_RGB_20220819-1230.tif",
    "KelpMap_S_20220818_MS_v2.tif": "KELPMAP_Vega_South_Spectrofly_MS_20220818-0000.tif",
    "1055rgb120_transparent_mosaic_group1.tif": "KELPMAP_Vega_North_NIVA_RGB_20220818-1055.tif",
    "0944_ms_120_composite.tif": "KELPMAP_Vega_North_NIVA_MS_20220819-0944.tif",
    "0814_rgb_115_transparent_mosaic_group1.tif": "KELPMAP_Vega_South_NIVA_RGB_20220819-0814.tif",
    "0814_ms_115_composite.tif": "KELPMAP_Vega_South_NIVA_MS_20220819-0814.tif",
    "1012_rgb_60_transparent_mosaic_group1.tif": "KELPMAP_Vega_South_NIVA_RGB_20220819-1012.tif",
}

cog_fold = r"/home/notebook/cogs/"
n_threads = 4

In [ ]:
# Check output names are unique
assert len(set(fnames_dict.values())) == len(fnames_dict.values())

In [ ]:
flist = []
for fname in fnames_dict.keys():
    fpaths = find_file_by_name(fname, base_dir)
    if len(fpaths) > 1:
        print(fpaths)
    elif len(fpaths) == 0:
        print("Could not find:", fname)
    else:
        flist.append(fpaths[0])
# flist

## 2. Convert to COGs

In [ ]:
for fpath in flist:
    fname = os.path.basename(fpath)
    cog_path = os.path.join(cog_fold, fnames_dict[fname])
    cmd = [
        "gdal_translate",
        "-b",
        "1",
        "-b",
        "2",
        "-b",
        "3",
        "-of",
        "COG",
        "-ot",
        "Byte",
        "-co",
        "COMPRESS=LZW",
        "-co",
        "PREDICTOR=2",
        "-co",
        f"NUM_THREADS={n_threads}",
        "-co",
        "OVERVIEWS=IGNORE_EXISTING",
        "-co",
        "BIGTIFF=YES",
        "-scale",
        "-a_nodata",
        "0",
        fpath,
        cog_path,
    ]
    subprocess.check_call(cmd)

## 3. Upload to GeoServer

In [ ]:
# Authernticate with GeoServer
geo = Geoserver(
    "https://geonode.seabee.sigma2.no/geoserver",
    username=SETTINGS.GEOSERVER_USER,
    password=SETTINGS.GEOSERVER_PASSWORD,
)

In [ ]:
# Upload COGs to GeoServer
workspace = "geonode"

search_path = os.path.join(cog_fold, "*.tif")
flist = glob(search_path)
for fpath in tqdm(flist):
    fname = os.path.basename(fpath)
    layer_name = os.path.splitext(fname)[0]

    # Add to GeoServer. Note: Will overwrite layer if it exists
    status = geo.create_coveragestore(
        layer_name=layer_name, path=fpath, workspace=workspace
    )
    # print(status)

## 4. Update GeoNode

Trigger the `updatelayers` command for each new layer via the GeoNode API.

Alternatively, this can be done manually by logging in to the GeoNode administration panel and navigatinge to

    Home > Management Commands Over HTTP > Management command jobs
    
Choose `Add management command job` and set the **Command** to `updatelayers`. Check the **Autostart** box and click **Save**. If you have added a lot of data, the update process may take a while. When it is finished, the status should be updated and the new images datasets be visible in GeoNode.

In [ ]:
base_url = "https://geonode.seabee.sigma2.no/api/v2/"
cmd_url = base_url + r"management/commands/"
status_url = base_url + r"management/jobs/"

headers = {"Content-Type": "application/json"}
auth = (SETTINGS.GEOSERVER_USER, SETTINGS.GEONODE_PASSWORD)

wait = 10  # seconds

search_path = os.path.join(cog_fold, "*.tif")
flist = glob(search_path)
for fpath in tqdm(flist):
    fname = os.path.basename(fpath)
    layer_name = os.path.splitext(fname)[0]

    # Update layer
    command = "updatelayers"
    kwargs = {"filter": layer_name, "store": layer_name, "workspace": "geonode"}
    response = requests.post(
        cmd_url,
        headers=headers,
        auth=auth,
        data=json.dumps({"command": command, "kwargs": kwargs}),
    )
    response.raise_for_status()

    # Optional: Wait for completion. Comment out this block to queue all
    # update operations. You can then just wait for the batch to finish
    # (e.g. check via the admin. panel)
    job_id = response.json()["data"]["id"]
    job_url = status_url + f"{job_id}/status/"
    job_status = "NOT_FINISHED"
    while job_status != "FINISHED":
        job_status = requests.get(job_url, auth=auth).json()["status"]
        time.sleep(wait)

## 5. Update metadata

In [ ]:
base_url = "https://geonode.seabee.sigma2.no/api/v2/"
headers = {"Authorization": f"Bearer {SETTINGS.GEONODE_TOKEN}"}
auth = (SETTINGS.GEOSERVER_USER, SETTINGS.GEONODE_PASSWORD)

inv_fnames_dict = {y: x for x, y in fnames_dict.items()}
search_path = os.path.join(cog_fold, "*.tif")
flist = glob(search_path)
for fpath in tqdm(flist):
    fname = os.path.basename(fpath)
    layer_name = os.path.splitext(fname)[0]

    # Find resource ID
    filter_url = base_url + f"resources?search={layer_name}&search_fields=title"
    response = requests.request("GET", filter_url, headers=headers)
    response.raise_for_status()
    data = response.json()
    assert data["total"] == 1, f"More than one dataset found with title '{layer_name}'."
    dataset_id = data["resources"][0]["pk"]

    # Extract metadata from names with format:
    #    project_region_area_org_spec_date-time.tif
    project, area, ns, org, bands, date = layer_name.split("_")
    date = dt.datetime.strptime(date, "%Y%m%d-%H%M")
    orig_fname = inv_fnames_dict[fname]
    abstract = (
        f"{bands} mosaic collected by {org} for the {project} project at {area} {ns} on {date}."
        f"<br><br><b>MinIO file name:</b> {orig_fname}."
    )

    # Update metadata
    data = {
        "abstract": abstract,
        "date": date.isoformat(),
        "date_type": "creation",
        "attribution": "SeaBee",
    }
    update_url = base_url + f"datasets/{dataset_id}"
    response = requests.patch(update_url, auth=auth, json=data)
    response.raise_for_status()